In [7]:
import numpy as np

def interval_projection(mu):
    if(mu < 0):
        mu = 0
    elif(mu >= 1):
        mu = 1
    return mu

# projects vector z onto all vectors living in [0,1]^f
def project_column(z):
    f = len(z)
    # initialize projection vector x
    x = np.zeros(f)   
    
    mu = z[0]
    
    kc = 0
    for k in range(1,f):
        if(z[k] <= interval_projection(mu)):
            kc = k - 1
            break
        else:
            mu = (k-1)/k * mu + 1/k * z[k]

    x[0] = interval_projection(mu)

    for k in range(1,kc):
        x[k] = interval_projection(mu)
    for k in range(kc, f):
        # assign the positive part of z[k]
        x[k] = z[k] if (z[k] > 0) else 0
        
    return x, mu
    
# helper function for hottopixx below
def project_on_phi(C):
    (f,f) = C.shape
    mu = np.zeros(f)
    for i in range(f):
        z = C[:,i]
        C[:,i], mu[i] = project_column(z)
    return C, mu

In [8]:
# X is the non-negative data matrix, f by n
# expects X's rows to sum to 1
# r is the dimension of the NMF, i.e. X = FW where F is f by r, W is r by n

def hottopixx(X,r):
    
    # get dimensions of data matrix (Careful: X is features times samples, different than usual)
    (f,n) = X.shape
    
    # primal and dual stepsizes
    sp = 1e-1
    sd = 1e-2

    # initialize cost vector randomly (does this have to be a positive vector?)
    p = np.random.rand(f)

    # 'one' vector
    one = np.ones(f)
    
    # mu vector
    mu = np.zeros(f)
    
    # C is f by f
    C = np.zeros((f,f))
    
    beta = 0
    
    # number of epochs
    N_ep = 3
    
    for t in range(N_ep):
        
        # do an epoch
        for i in range(n):
            
            # choose random column of data matrix
            k = math.floor(np.random.rand() * n)
            Xk = X[:,k]
            
            # term in the sign function (see paper)
            term1 = np.sign(Xk - np.dot(C,Xk))
            
            # incremental gradient descent step
            C = C + sp * np.dot(term1, Xk.T) - sp * np.diag(mu * (beta * one - p))
            
        # project C onto Phi_0
        C, mu = project_on_phi(C)
        
        # update beta
        beta = beta + sd * (np.trace(C) - r)
    
    
    # now, we have selected the most salient features of the data matrix.
    # Let's extract them:
    # - Implement
    
    # Solve for F with linear program 